In [ ]:
#from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import bs4 as bs
import time
import requests
import pandas as pd
#pretty print package
from pprint import pprint
from pathlib import Path

In [ ]:
# Initializing the webdriver
options = webdriver.ChromeOptions()
service = Service(executable_path="/Users/shivani/anaconda3/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
url = 'https://www.glassdoor.com/Job/germany-data-scientist-jobs-SRCH_IL.0,7_IN96_KO8,22.htm?clickSource=searchBox'
gs_html = driver.get(url)
time.sleep(4)
pageSource = driver.page_source

In [ ]:
pageSource

In [ ]:
#Test for the "Sign Up" prompt and get rid of it.
try:
    driver.find_element(by=By.CLASS_NAME,value="selected").click()
except ElementClickInterceptedException:
    pass

time.sleep(.1)

try:
    driver.find_element(by=By.XPATH, value='/html/body/div[12]/div/div/div[2]/span').click()  #clicking to the X.
except NoSuchElementException:
    pass

In [ ]:
#To accept cookies
try:
    driver.find_element(by=By.ID,value='onetrust-accept-btn-handler').click()
except: 
    pass

In [ ]:
source = driver.page_source

In [ ]:
#Saving source page into html 
fileToWrite = open("page_source.html", "w")
fileToWrite.write(pageSource)
fileToWrite.close()
fileToRead = open("page_source.html", "r")
pprint(fileToRead.read())
fileToRead.close()
driver.quit()

In [ ]:
with open("page_source.html", "r") as file:
    gs_jobs= file.read()
gs_jobs

In [ ]:
# Create soup object
soup = bs.BeautifulSoup(gs_jobs, "html.parser")
#base_url = "https://www.glassdoor.com/"
soup

In [ ]:
def get_urls_from_html(gs_jobs):
    """
    Extract Glassdoor job posting URLs from HTML
    """
    
    # Create soup object
    soup = bs.BeautifulSoup(gs_jobs, "html.parser")
    base_url = "https://www.glassdoor.com"

    ul = soup.find("ul", {"data-test": "jlGrid"})

    expose_urls = []
    children = ul.find_all("li", {"class": "react-job-listing"})

    for child in children:
        for link in child.find_all("a"):
            expose_urls.append(base_url + link['href'])
    
    return expose_urls

In [ ]:
expose_urls = get_urls_from_html(gs_jobs)
expose_urls

In [ ]:
#To scrape the data from each link

job_links=[]
for index,url in enumerate(expose_urls):
    pprint(url)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    response = driver.page_source
    job_links.append(response)
    #pprint(response)
    
    with open(f"./data/scraped_links/job{index}.htm", "w") as f:
       f.write(response)
    
    driver.quit()


In [ ]:
#extracting details from the links

def parse_html(html):
    
    soup = bs.BeautifulSoup(html, "html.parser")

    title = soup.find('div', {'data-test':'job-title'}).get_text()

    location = soup.find("span", {'data-test':'location'}).get_text()

    company_name = soup.find("div", class_="d-flex align-items-center justify-content-between css-16nw49e e11nt52q1").get_text()

    job_description = soup.find("div", id="JobDescriptionContainer").get_text()
    
    jobs= {
        "title": title,
        "location": location,
        "company_name": company_name,
        "job_description": job_description
    }

    return jobs

In [ ]:
#read the path for links and create a data frame
dir_html = Path('./data/scraped_links/')
df = pd.DataFrame(columns=["title", "location", "company_name", "job_description"])

for file in dir_html.iterdir():
    with open(file, "r") as f:
       html = f.read()
    
    data = parse_html(html)
    df.loc[len(df)] = data.values()

df

In [ ]:
#Save the data frame into csv file
df.to_csv('./data/glassdoor_jobs.csv', index=False)